In [ ]:
import torch 
import torch.nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

from Trainer import Trainer

from variational_probing import VariationalProbingModel
from utils import *

from transformers import AutoTokenizer

In [3]:
model = VariationalProbingModel(pretrained_path="D:/models/roberta-base").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("D:/models/roberta-base")

Some weights of RobertaModel were not initialized from the model checkpoint at D:/models/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\anaconda3\envs\DL\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
inps = tokenizer(["hello world!"], return_tensors="pt")

In [9]:
test_output = model(inps["input_ids"].to("cuda"), inps["attention_mask"].to("cuda"))

In [10]:
test_output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0710,  0.0783,  0.0031,  ..., -0.0746, -0.0541,  0.0182],
         [-0.1743, -0.2809, -0.0731,  ..., -0.3774,  0.0176,  0.3123],
         [-0.1348,  0.0859,  0.1506,  ..., -0.1907, -0.1079,  0.3542],
         [-0.2056, -0.2489,  0.2531,  ...,  0.0606, -0.2765,  0.7178],
         [-0.0636,  0.0742, -0.0212,  ..., -0.1239, -0.0633,  0.0021]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-3.5100e-01, -4.8025e-01, -3.6435e-01, -2.2246e-01,  1.5880e-01,
          1.6888e-01, -1.4584e-01,  3.8146e-01, -1.6976e-01, -7.0282e-01,
          2.0111e-01,  3.9626e-02, -1.0630e-01, -5.3962e-01,  1.6174e-01,
         -2.6259e-01,  2.3962e-01, -1.0403e-01, -3.8447e-03,  3.0312e-02,
          1.1269e-01, -3.9087e-01, -2.8602e-01,  4.8452e-01, -4.2954e-01,
         -1.8140e-01,  5.0511e-01,  4.2459e-01,  5.0607e-01,  2.0992e-01,
         -3.6054e-01, -5.3043e-01,  5.8689e-01, -6.3716e-01, 

In [ ]:
trainer = Trainer(
    model=model, 
    train_config={
        "variational": True, 
        "eval_metrics": ["description_length"],
        "lr": 1e-3,
        "optimizer": "Adam",
        "n_epochs": 100,
        "loss_function": "crossentropy"
    }
)

data_path_train = ...
data_path_val = ...
data_path_test = ...


train_dataset = MDLDataset_POSTagging(data_path_train)
val_dataset = MDLDataset_POSTagging(data_path_val)
test_dataset = MDLDataset_POSTagging(data_path_test)

collator = Collator(
    tokenizer=tokenizer,
    max_length=1024,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collator)

val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collator)

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collator)


In [ ]:

model = trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    evaluate_every=10
)

val_metrics_mean = trainer._metrics

In [ ]:
test_metrics_mean, test_metrics = trainer.evaluate(test_loader)

In [ ]:
def visualize(metrics_mean): # TODO: implement and transfer to utils
    ...


visualize(val_metrics_mean)